# Calculation of elementary growth modes (EGMs) & elementary growth vectors (EGVs) for a small growth model.
## Constraints:  
* capacity constraints for all catalysts (importers, enzymes, and the ribosome)
* membrane area constraint
* minimum fraction of the surface area formed by lipids

In [1]:
import efmtool
import numpy as np
import pandas as pd
from itertools import compress

In [2]:
def make_nice_results(egms, rxns, drop_slack = True):
    """Convert output of efmtool to a data frame
    normalize by the C (constraint) column
    if drop_slack = True, drop columns with slack variables and constraints (named S* or C)"""
    
    # solve "ValueError: Big-endian buffer not supported on little-endian compiler"
    egms = egms.byteswap().newbyteorder()

    res = pd.DataFrame(egms.T, index = ["EGM%s" % (i+1) for i in range(egms.shape[1])], columns = rxns)

    # normalize values by the column C
    if "C" in res.columns:
        res = res.div(res.C, axis = 0)
        
    if drop_slack:
        cols = [c for c in res.columns if c.startswith("S")]
        if "C" in res.columns:
            cols = cols+["C"]
        res = res[res.columns.drop(cols)]
    return res


def egms_to_binary(egms, sort = False):
    """Convert EGMs to binary, then to strings. 
    sort them alphabetically (if sort=True)"""

    bin_egms = egms.astype(bool).astype(int)
    efm_types = []
    for index, row in bin_egms.iterrows():
        efm_types.append("".join(row.astype(str)))
        
    if sort:
        efm_types.sort()
    
    return efm_types


def rxns_from_bool(rxns, egm_str):
    """Get a list of reactions from a string of bools
    Example input: rxns = [a, b, c], egm_str = "110" 
    Example output: [a, b]
    """

    active = [bool(int(letter)) for letter in egm_str]
    return list(compress(rxns, active))
    

Parameters used in the model (links)
* ```nL``` - ~N Glc in lipids (in terms of carbons)] - estimated from the [most common membrane lipid - PE](https://www.ncbi.nlm.nih.gov/books/NBK26871/), [most common fatty acid - 16:0](https://bionumbers.hms.harvard.edu/bionumber.aspx?id=109367&ver=1&trm=fatty+acid&org=)
* ```nR``` - [AA in ribosome](https://bionumbers.hms.harvard.edu/bionumber.aspx?id=110218&ver=7&trm=ribosome+amino+acid&org=)
* ```nI``` - [AA in glucose transporter](https://biocyc.org/META/NEW-IMAGE?type=ENZYME&object=CPLX-157)
* ```nE``` - [AA in a typical protein](https://bionumbers.hms.harvard.edu/bionumber.aspx?id=108986&ver=2&trm=protein+length&org=)
* ```k_cat``` - [k_cat (central carbon metabolism)](https://doi.org/10.1021/bi2002289)
* ```k_el``` - [Translation rate](https://bionumbers.hms.harvard.edu/bionumber.aspx?id=111689&ver=2&trm=translation+rate&org=)
* ```density``` - [Dry mass per unit cell volume (density)](https://bionumbers.hms.harvard.edu/bionumber.aspx?id=104443&ver=5&trm=dry+mass+volume&org=)
* ```AI``` - [Area of a glucose transporter dimer](https://bionumbers.hms.harvard.edu/bionumber.aspx?id=114685&ver=3&trm=transporter&org=)
* ```AL``` - [Area of a lipid](https://bionumbers.hms.harvard.edu/bionumber.aspx?id=114186&ver=0&trm=lipid+surface&org=)
* [S/V ratio](https://doi.org/10.1016/j.tim.2018.04.008) - S/V = 9.3-2.8*mu
* ```alpha``` - [minimum lipid fraction in the membrane](https://www.ncbi.nlm.nih.gov/books/NBK26871/)

In [3]:
# stoichiometry
nL = 7   
nR = 7536*3  # multiplied by 3 to take into account the RNA mass of the ribosome
nI = 646
nE = 325

# kinetic parameters [1/h]
k_cat = 79*3600
k_el = 8*3600

# parameters for area constraint
density = 290  # g/L
unit_conv = 1/(density*6.022*10**20*10**(-15))  # unit conversion for area constraint
AI = 48*10**-6  # um^2
AL = 0.5*10**-6  # um^2
alpha = 0.5

# dry mass constraint
MW_Glc = 0.18  # glucose MW [g/mmol]
MW_NH4 = 0.018  # ammonium MW [g/mmol]

# all molecular masses
MW_AA = MW_Glc+MW_NH4
mw = [MW_Glc, MW_NH4, MW_AA, 
      nL*MW_Glc, nL*MW_Glc, 
      nI*MW_AA, nI*MW_AA, 
      nE*MW_AA, nE*MW_AA, nE*MW_AA, nR*MW_AA]

## Define stoichiometric matrices
* without constraints (```original matrix```) 
* with constraints (```S```) for every growth rate

In [4]:
# range of growth rates to test
mus = np.linspace(0.005,1.3,1000)
mets_original = ["G", "N", "AA", "LD", "L", "IG", "IN", "EAA", "ELD", "EL", "R"]
rxns = ["IG", "IN", "EAA", "ELD", "EL", "R_IG", "R_IN", "R_EAA", "R_ELD", "R_EL", "R_R", 
        "S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "S10", "S11", "C"]

# matrix without constraints
original_matrix = np.array([[1, 0, -1, -nL, 0, 0, 0, 0, 0, 0, 0],
                          [0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 1, 0, 0, -nI, -nI, -nE, -nE, -nE, -nR],
                          [0, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
original_matrix = pd.DataFrame(original_matrix, index = mets_original, columns=rxns[:11])

# matrices with constraints on enzyme capacity, area and uptakes
row_names = ["G", "N", "AA", "LD", "C1", "C2", "C3", "C4", "C5", "C6", "A", "U", "LF"]
matrixes = {}
mus_dict = {}
for mu in mus:
    S = np.array([[1, 0, -1, -nL, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, -nI, -nI, -nE, -nE, -nE, -nR, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0],
                  [-1, 0, 0, 0, 0, k_cat/mu, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0],
                  [0, -1, 0, 0, 0, 0, k_cat/mu, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
                  [0, 0, -1, 0, 0, 0, 0, k_cat/mu, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0],
                  [0, 0, 0, -1, 0, 0, 0, 0, k_cat/mu, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0],
                  [0, 0, 0, 0, -1, 0, 0, 0, 0, k_cat/mu, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
                  [0, 0, 0, 0, 0,-nI,-nI,-nE,-nE,-nE, -nR+k_el/mu, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0],
                  [0, 0, 0, 0, AL-AL*alpha, -AI*alpha, -AI*alpha, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
                  [0, 0, 0, 0, AL, AI, AI, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -unit_conv*mu*(9.3-2.8*mu)],
                  [MW_Glc, MW_NH4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -mu]])
    matrixes[str(round(mu,4))] = S
    mus_dict[str(round(mu,4))] = mu

# reversibilities - all irreversible
revs = [0] * S.shape[1]

## Calculate EGMs and concentrations

In [5]:
options = efmtool.get_default_options()
options["arithmetic"] = "fractional"

In [6]:
# calculate elementary growth modes
matrix_wo_constraints = S[:4, :15]  # take the last "S", these rows are the same at all mus
egms = efmtool.calculate_efms(stoichiometry = matrix_wo_constraints, 
                              reversibilities = revs[:15], 
                              reaction_names = rxns[:15], 
                              metabolite_names = row_names[:4],
                              options = options)
egms_nice = make_nice_results(egms, rxns[:15], True)
egms_nice

,IG,IN,EAA,ELD,EL,R_IG,R_IN,R_EAA,R_ELD,R_EL,R_R
EGM1,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EGM2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EGM3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EGM4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EGM5,7.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
EGM6,646.0,646.0,646.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
EGM7,646.0,646.0,646.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
EGM8,325.0,325.0,325.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
EGM9,325.0,325.0,325.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
EGM10,325.0,325.0,325.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# calculate concentrations
egms_conc = pd.DataFrame(columns = mets_original, index = egms_nice.index)
for egm in egms_nice.index:
    Nv = original_matrix.multiply(egms_nice.loc[egm]).sum(axis=1)
    egms_conc.loc[egm] = Nv/sum(mw*Nv)
egms_conc = egms_conc.multiply(mw, axis=1)
egms_conc

,G,N,AA,LD,L,IG,IN,EAA,ELD,EL,R
EGM1,0,0,0,1,0,0,0,0,0,0,0
EGM2,1,0,0,0,0,0,0,0,0,0,0
EGM3,0,1,0,0,0,0,0,0,0,0,0
EGM4,0,0,1,0,0,0,0,0,0,0,0
EGM5,0,0,0,0,1,0,0,0,0,0,0
EGM6,0,0,0,0,0,1,0,0,0,0,0
EGM7,0,0,0,0,0,0,1,0,0,0,0
EGM8,0,0,0,0,0,0,0,1,0,0,0
EGM9,0,0,0,0,0,0,0,0,1,0,0
EGM10,0,0,0,0,0,0,0,0,0,1,0


## Calculate EGVs and concentrations (with constraints)

In [8]:
nice_egvs_mu_all = {}
cut_points = {}
N_egvs = 0
for mu in matrixes:
    egvs = efmtool.calculate_efms(stoichiometry = matrixes[mu], reversibilities = revs, 
                                  reaction_names = rxns, metabolite_names = row_names,
                                  options = options)
    # print when the number of EGVs changes
    if N_egvs != egvs.shape[1]:
        print("From growth rate %s: %s EGVs" % (mu, egvs.shape[1]))
        N_egvs = egvs.shape[1]

    # save data
    if egvs.shape[1] != 0:
        nice_egvs_mu_all[mu] = make_nice_results(egvs, rxns, False)
    
    # make categories according to the number of modes
    try:
        cut_points[egvs.shape[1]].append(mu)
    except KeyError:
        cut_points[egvs.shape[1]] = [mu]

From growth rate 0.005: 24 EGVs
From growth rate 1.2131: 10 EGVs
From growth rate 1.265: 0 EGVs


## Check if EGVs are consistent & name them
There are two phases - 24 EGVs up to some growth rate, then 10 EGVs. Check if the EGVs have the same support for all growth rates within each phase.

In [9]:
# reference modes for the lowest growth rate with 10 modes
reference10 = egms_to_binary(nice_egvs_mu_all[cut_points[10][0]], sort = True)

print(f"Number of modes: {len(reference10)}")
for r in reference10:
    print(r, rxns_from_bool(rxns, r))

Number of modes: 10
11111111111000000000111 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S10', 'S11', 'C']
11111111111000000001011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S9', 'S11', 'C']
11111111111000000010011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S8', 'S11', 'C']
11111111111000000100011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S7', 'S11', 'C']
11111111111000001000011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S6', 'S11', 'C']
11111111111000010000011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S5', 'S11', 'C']
11111111111000100000011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S4', 'S11', 'C']
11111111111001000000011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S3',

In [10]:
# reference modes for the lowest growth rate with 24 modes
reference24 = egms_to_binary(nice_egvs_mu_all[cut_points[24][0]], sort = True)
        
print(f"Number of modes: {len(reference24)}")
for r in reference24:
    print(r, rxns_from_bool(rxns, r))

Number of modes: 24
11111111111000000000111 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S10', 'S11', 'C']
11111111111000000001011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S9', 'S11', 'C']
11111111111000000010011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S8', 'S11', 'C']
11111111111000000100011 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S7', 'S11', 'C']
11111111111000001000101 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S6', 'S10', 'C']
11111111111000001001001 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S6', 'S9', 'C']
11111111111000001010001 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S6', 'S8', 'C']
11111111111000001100001 ['IG', 'IN', 'EAA', 'ELD', 'EL', 'R_IG', 'R_IN', 'R_EAA', 'R_ELD', 'R_EL', 'R_R', 'S6', '

In [11]:
# order the EGVs
naming_dict = {"11111111111100000000011": "EGV1",
              "11111111111000100000011": "EGV2",
              "11111111111010000000011": "EGV3",
              "11111111111001000000011": "EGV4",
              "11111111111000000100011": "EGV5",
              "11111111111000000010011": "EGV6",
              "11111111111000000001011": "EGV7",
              "11111111111000000000111": "EGV8",
              "11111111111100010000001": "EGV9",
              "11111111111000110000001": "EGV10",
              "11111111111010010000001": "EGV11",
              "11111111111001010000001": "EGV12",
              "11111111111000010100001": "EGV13",
              "11111111111000010010001": "EGV14",
              "11111111111000010001001": "EGV15",
              "11111111111000010000101": "EGV16",
              "11111111111100001000001": "EGV17",
              "11111111111000101000001": "EGV18",
              "11111111111010001000001": "EGV19",
              "11111111111001001000001": "EGV20",
              "11111111111000001100001": "EGV21",
              "11111111111000001010001": "EGV22",
              "11111111111000001001001": "EGV23",
              "11111111111000001000101": "EGV24",
              "11111111111000010000011": "EGV25",
              "11111111111000001000011": "EGV26"}

In [12]:
references = {10: reference10, 24: reference24}

In [13]:
# check if the the type of modes are the same for all growth rates with the same number of modes
n_diff = 0
for egv in nice_egvs_mu_all:
    binary = egms_to_binary(nice_egvs_mu_all[egv])
    ref = references[len(binary)]  # take the reference with the same amount of modes
    diff = set(binary).difference(set(ref))
    if len(diff) > 0:
        print(f" mu: {egv}; N modes: {len(binary)}; Not in reference: {diff}")
        n_diff += 1

if n_diff == 0:
    print("All modes within blocks are consistent")

All modes within blocks are consistent


## Process and save data

In [14]:
# merge results in one table and save
all_results = pd.DataFrame(columns=rxns)
for mu in nice_egvs_mu_all:
    res = nice_egvs_mu_all[mu]
    res["mu"] = mus_dict[mu]  # save the float mu
    res["mu_str"] = mu
    all_results = all_results.append(res)

binary = egms_to_binary(all_results, sort = False)

# number EGVs accoring to the naming dictionary
all_results["EGVs"] = [naming_dict[egv[:-2]] for egv in binary]
all_results = all_results.set_index([all_results["mu_str"]+all_results["EGVs"]])

all_results.to_csv("../data/EGVs_LDs_min_lip.csv")

## Calculate concentrations and save

In [15]:
# calculations of concentrations - with x = NV/mu (done with the original matrix!)
all_concentrations = pd.DataFrame(columns = mets_original, index = all_results.index)
for egv in all_results.index:
    mu = all_results.loc[egv, "mu"]
    all_concentrations.loc[egv] = original_matrix.multiply(all_results.loc[egv][:11]).sum(axis=1)/mu
all_concentrations["mu"] = all_results.mu
all_concentrations["mu_str"] = all_results.mu_str
all_concentrations["EGVs"] = all_results.EGVs

all_concentrations.to_csv("../data/concentrations_LDs_min_lip_mmol.csv")
# convert to g/gDW
all_concentrations.loc[:,mets_original] = all_concentrations.loc[:,mets_original].multiply(mw, axis=1)
all_concentrations.to_csv("../data/concentrations_LDs_min_lip.csv")